# This file runs study 1, no winner case, with social network

In [1]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import repeat
from scipy import stats
import seaborn as sns
import pandas as pd
import scipy.stats as st
import scipy.stats as st
def best_coverage(data,n_trials,n_resamples,true_val):
    xb = np.random.choice(data[:,0], (n_resamples, n_trials), replace=True)
    low,high=st.t.interval(0.95, len(xb)-1, loc=np.mean(xb,0), scale=st.sem(xb,0))
    print(low,high,true_val)
    return np.sum(np.logical_and(true_val >= low, true_val <= high))/n_trials


In [2]:
class network_MAB:
    def __init__(self, probs, karms, nfirst, nEpisodes, nets, augment):
        '''
        meanings for the parameters:
        probs: True success rate for each arm k in {1, ..., K} 
        kArms: Number of arms to choose among
        nsamples: Number of subjects to test at each time step
        random_seed: Store the number of seed for replication

        '''
 
        self.network = nets
        self.sample_indexs = list(range(4039))
        self.selection = []
        self.left = list(range(4039))
        self.neighbors = set()
        self.augment = augment

        self.n_obs = 4000
        self.probs = probs
        self.K = karms
        self.n = int((self.n_obs-nfirst)/(nEpisodes-1)) if nEpisodes > 1 else 0
        self.T = nEpisodes
        self.first = nfirst
        self.true_win_arm = np.argmax(np.asarray(probs))


    def best_arm(self, s, asgn):
        draws = 10000
        # + 1 for a Beta(1, 1) prior
        new_alpha = s+1
        new_beta = asgn-s+1
        selection_table = []
        for i in range(self.K):
            theta = np.random.beta(new_alpha[i], new_beta[i], draws)
            selection_table.append(theta)

        winning_arms = np.argmax(selection_table, axis = 0)
        winning_prob = []

        c = Counter(winning_arms)
        winning_prob = []
        for i in range(self.K):
            winning_prob.append(c[i]/draws)

        return np.array(winning_prob)

    def experiment(self, winning_probs, sampleSize, static = False):  
        # update sample information
        rest = self.left
        x = np.random.choice(rest, size=sampleSize, replace=False)
        self.selection.append(x)
        self.left = list(set(rest) - set(x))   

        # random assignment

        if not static:
            # print(type(self.n),type(winning_probs))
            new_assign = np.random.choice(range(self.K), size=sampleSize, p=list(winning_probs))
        else:
            new_assign = np.random.choice(range(self.K), size=sampleSize)
        # count number of assigned subjects
        c = Counter(new_assign)
        assigned = [c[i] for i in range(self.K)]

        success = [0]*self.K
        # print('self.probs',self.probs, '\n new assign',new_assign)
        # new_assign: array of group assignment of each sample

        # do experiment
        for i in range(len(x)):
            if x[i] in self.neighbors:
                success_probs = self.probs[new_assign[i]]+ self.augment
                success[new_assign[i]] += np.random.binomial(1,success_probs,1)[0]
            else:
                success_probs = self.probs[new_assign[i]]
                success[new_assign[i]] += np.random.binomial(1,success_probs,1)[0]
            
        # update information again
        new_neighbors = []
        for i in x:
            new_neighbors += list(nx.classes.function.all_neighbors(G1,i))      
        
        self.neighbors.update(set(new_neighbors))
        # print(success)
        return (assigned, success)

    def reset(self):
    # initialization
        success = np.zeros((self.T, self.K))   # record the success cases of each arm at each time step
        assigned = np.zeros((self.T, self.K)) # record the assigned cases of each arm at each time step
        posterior_probs = 0 # estimated success rate
        winning_probs = np.zeros((self.T, self.K)) # probability to be the true best arm
        # rewards = np.zeros(self.T) # record the average rewards per period
        # regrets = np.zeros(self.T) # record the regrets per period
        self.selection = []
        self.left = list(range(4039))
        self.neighbors = set()
        return (success, assigned, posterior_probs, winning_probs)

    def Thompson_sampling(self):
        # initialization
        (success, assigned, posterior_probs, winning_probs) = self.reset()

        # for round 1, samples are randomly assigned
        # new_assign = np.random.choice(range(self.K), size=self.first)
        assigned[0], success[0]  = self.experiment(1/self.K,self.first,static=True)
        # print(success[0,0],assigned[0,0])
        posterior_probs = success[0,0]/ assigned[0,0]
        winning_probs[0] = self.best_arm(success[0], assigned[0])
        adj_probs = success[0][0]*self.K/self.first
        # print(adj_probs)
        if self.T > 1:
            count0 = 0
            for i in range(1, self.T):              
                # new_assign = np.random.choice(range(self.K), size=self.n, p=list(winning_probs[i-1])) # adaptive design
                new_assigned, new_success = self.experiment(winning_probs[i-1],self.n)
                assigned[i] = assigned[i-1]+ new_assigned
                success[i] = success[i-1]+ new_success
                winning_probs[i] = self.best_arm(success[i], assigned[i])
                if winning_probs[i-1,0] == 0:
                    count0 += 1
                else:
                    adj_probs += new_success[0]/winning_probs[i-1,0]/self.n
            posterior_probs = adj_probs / (self.T-count0)
        return (success, assigned, posterior_probs, winning_probs)
    


    def static(self):
        # initialization
        (success, assigned, posterior_probs, winning_probs) = self.reset()

        # for round 1, samples are randomly assigned
        # new_assign = np.random.choice(range(self.K), size=self.first).tolist()
        assigned[0], success[0]  = self.experiment(1/self.K,self.first,static=True)
        posterior_probs = success[0,0]/ assigned[0,0]
        winning_probs[0] = self.best_arm(success[0], assigned[0])
        adj_probs = success[0][0]*self.K/self.first
        for i in range(1, self.T):
            # new_assign = np.random.choice(range(self.K), size=self.n).tolist() # static design
            new_assigned, new_success = self.experiment(1/self.K, self.n, static = True)
            assigned[i] = assigned[i-1]+ new_assigned
            success[i] = success[i-1]+ new_success
            winning_probs[i] = self.best_arm(success[i], assigned[i])
            adj_probs += new_success[0]*self.K/self.n
        posterior_probs = adj_probs / self.T


        return (success, assigned, posterior_probs, winning_probs)
        
    def do_replication(self, times, method):
        '''
        parameters:
        final_regrets -- to record the final regret of each replication, in order to compare efficiency
        final_win_arm -- to record the final win arm selected by two methods, in order to compare the accurarcy
        final_win_probs -- to record the final probability of each arm being the best arm, in order to compare the accurarcy
        final_assignment -- to record the final number of assigned subjects to the true best arm, comparing the exploitition
        cum_rewards -- to record the total rewards of every replication
        '''
        # records of the replication
        final_win_arm = np.zeros(times)
        final_win_probs = np.zeros(shape = (times, self.K))
        final_assignment = np.zeros(times) # only record the true best arm
        estimation = np.zeros(times)
        
        if method == 'TS':
            for i in range(times):
                (success, assigned, posterior_probs, winning_probs) = self.Thompson_sampling()
                final_win_arm[i] = np.argmax(winning_probs[-1])
                final_win_probs[i] = winning_probs[-1]
                final_assignment[i] = assigned[-1][self.true_win_arm]
                estimation[i] = posterior_probs
        else:
            for i in range(times):
                (success, assigned, posterior_probs, winning_probs) = self.static()
                final_win_arm[i] = np.argmax(winning_probs[-1])
                final_win_probs[i] = winning_probs[-1]
                final_assignment[i] = assigned[-1][self.true_win_arm]
                estimation[i] = posterior_probs

        ate = np.mean(estimation)
        mse = np.mean(np.square(np.subtract(estimation,self.probs[0])))
        rmse = np.sqrt(mse)
        # rmse = np.std(estimation,0)
        # coverage = best_coverage(estimation, 5000,100,self.probs[0])
        low,high=st.t.interval(0.95, len(estimation)-1, loc=np.mean(estimation), scale=st.sem(estimation))
        win_counts = Counter(final_win_arm)
        estimate = {"best_selected":win_counts[0]/times,"ATE":ate, "RMSE":rmse, "CI":(low,high)}
        print(estimate)
        return (final_win_arm, final_win_probs, final_assignment, estimation)
    

In [3]:
%pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [4]:
import networkx as nx

In [5]:
G1 =nx.read_edgelist("facebook_combined.txt", create_using = nx.Graph(), nodetype=int)
neigh = [1,20,40,65,75,90,1000]
for i in range(len(neigh)):
    all_neighbors = list(nx.classes.function.all_neighbors(G1,neigh[i]))
    print("All neighbors for Node ", str(neigh[i])," ---> ", str(all_neighbors))

All neighbors for Node  1  --->  [0, 48, 53, 54, 73, 88, 92, 119, 126, 133, 194, 236, 280, 299, 315, 322, 346]
All neighbors for Node  20  --->  [0, 2, 14, 41, 44, 111, 115, 149, 162, 214, 226, 312, 326, 333, 343]
All neighbors for Node  40  --->  [0, 21, 25, 26, 29, 56, 67, 72, 77, 113, 132, 133, 141, 142, 158, 169, 172, 199, 200, 203, 212, 213, 224, 231, 232, 239, 257, 258, 265, 271, 272, 274, 277, 280, 298, 304, 307, 315, 317, 322, 325, 329, 332, 334]
All neighbors for Node  65  --->  [0, 7, 13, 25, 82, 118, 203, 252, 261, 297, 314, 339]
All neighbors for Node  75  --->  [0, 9, 56, 67, 85, 170, 188, 200, 258, 272, 274, 304, 322, 323]
All neighbors for Node  90  --->  [0, 179]
All neighbors for Node  1000  --->  [107, 924, 974, 985, 1010, 1127, 1134, 1228, 1304, 1474, 1640, 1667, 1703, 1725, 1759, 1840]


In [6]:
k = 9 # number of treatments
np.random.seed(99332)
probs = [0.11] +[0.1]*8 # true value

In [7]:
first = 2000
periods = 2
# experiment 1
sim1 = network_MAB(probs,k,first, periods,G1,0.2)
(s1_ts_final_win_arm, s1_ts_final_win_probs, s1_ts_final_assignment, s1_ts_estimation) = sim1.do_replication(5000, "TS")
(s1_st_final_win_arm, s1_st_final_win_probs, s1_st_final_assignment, s1_st_estimation) = sim1.do_replication(5000, "static")

{'best_selected': 0.246, 'ATE': 0.2080437416369117, 'RMSE': 0.11676688297442082, 'CI': (0.2062852960920254, 0.209802187181798)}
{'best_selected': 0.2232, 'ATE': 0.2081358, 'RMSE': 0.10033210129365376, 'CI': (0.20755690102016655, 0.20871469897983347)}


In [8]:
first = 800
periods = 5
# experiment 2
sim2 = network_MAB(probs,k,first, periods,G1,0.2)
(s2_ts_final_win_arm, s2_ts_final_win_probs, s2_ts_final_assignment, s2_ts_estimation) = sim2.do_replication(5000, "TS")
(s2_st_final_win_arm, s2_st_final_win_probs, s2_st_final_assignment, s2_st_estimation) = sim2.do_replication(5000, "static")

{'best_selected': 0.2514, 'ATE': 0.23820622319623652, 'RMSE': 0.26205455761743085, 'CI': (0.2318690492633882, 0.24454339712908485)}
{'best_selected': 0.2068, 'ATE': 0.2646144, 'RMSE': 0.15645184714473653, 'CI': (0.26395150375437637, 0.2652772962456237)}


In [9]:
first = 400
periods = 10
# experiment 3
sim3 = network_MAB(probs,k,first, periods,G1,0.2)
(s3_ts_final_win_arm, s3_ts_final_win_probs, s3_ts_final_assignment, s3_ts_estimation) = sim3.do_replication(5000, "TS")
(s3_st_final_win_arm, s3_st_final_win_probs, s3_st_final_assignment, s3_st_estimation) = sim3.do_replication(5000, "static")


{'best_selected': 0.2324, 'ATE': 0.25941593051143774, 'RMSE': 0.31300556405583085, 'CI': (0.25178972361775787, 0.2670421374051176)}
{'best_selected': 0.2022, 'ATE': 0.28137510000000004, 'RMSE': 0.1731365802625199, 'CI': (0.2806920499099799, 0.28205815009002017)}


In [10]:
first = 200
periods = 20
# experiment 4
sim4 = network_MAB(probs,k,first, periods,G1,0.2)
(s4_ts_final_win_arm, s4_ts_final_win_probs, s4_ts_final_assignment, s4_ts_estimation) = sim4.do_replication(5000, "TS")
(s4_st_final_win_arm, s4_st_final_win_probs, s4_st_final_assignment, s4_st_estimation) = sim4.do_replication(5000, "static")


{'best_selected': 0.232, 'ATE': 0.2806799592281511, 'RMSE': 0.24890004507003413, 'CI': (0.27565678522327564, 0.28570313323302654)}
{'best_selected': 0.21, 'ATE': 0.28985535, 'RMSE': 0.18159440619826375, 'CI': (0.28916017769277524, 0.2905505223072248)}


In [11]:
first = 40
periods = 100
# experiment 5
sim5 = network_MAB(probs,k,first, periods,G1,0.2)
(s5_ts_final_win_arm, s5_ts_final_win_probs, s5_ts_final_assignment, s5_ts_estimation) = sim5.do_replication(5000, "TS")
(s5_st_final_win_arm, s5_st_final_win_probs, s5_st_final_assignment, s5_st_estimation) = sim5.do_replication(5000, "static")


<ipython-input-2-a347d553d3bd>:109: RuntimeWarning: invalid value encountered in double_scalars
  posterior_probs = success[0,0]/ assigned[0,0]


{'best_selected': 0.2252, 'ATE': 0.29573348262433463, 'RMSE': 0.20699868944853786, 'CI': (0.29319955193072833, 0.29826741331794093)}


<ipython-input-2-a347d553d3bd>:137: RuntimeWarning: invalid value encountered in double_scalars
  posterior_probs = success[0,0]/ assigned[0,0]


{'best_selected': 0.2054, 'ATE': 0.29511495000000015, 'RMSE': 0.18686531637117693, 'CI': (0.29440743424061055, 0.29582246575938975)}


In [12]:
first = 80
periods = 50
# experiment 6
sim6 = network_MAB(probs,k,first, periods,G1,0.2)
(s6_ts_final_win_arm, s6_ts_final_win_probs, s6_ts_final_assignment, s6_ts_estimation) = sim6.do_replication(5000, "TS")
(s6_st_final_win_arm, s6_st_final_win_probs, s6_st_final_assignment, s6_st_estimation) = sim6.do_replication(5000, "static")


<ipython-input-2-a347d553d3bd>:109: RuntimeWarning: invalid value encountered in double_scalars
  posterior_probs = success[0,0]/ assigned[0,0]


{'best_selected': 0.236, 'ATE': 0.29265944419625556, 'RMSE': 0.2046991758689836, 'CI': (0.29009748947010516, 0.29522139892240595)}
{'best_selected': 0.1996, 'ATE': 0.29360295000000003, 'RMSE': 0.1853844333742723, 'CI': (0.2928920502980922, 0.2943138497019079)}
